In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
 
from torch.utils.data import DataLoader, Dataset 
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import random
import numpy as np
import torch 
import os 
from tqdm import tqdm
from sklearn.metrics import f1_score
import polars
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

set_seed(42)

In [2]:
dict_ = {
    #'unavailable': 0,
    'no_relevant': 0,
    'relevant_minus': 1,
    'relevant': 2,
    'relevant_plus': 3,
    #'doubles': 5
}

In [3]:
items = polars.read_parquet('/kaggle/input/path-to-dzen/items.parquet', columns=['content', 'title', 'itemId']).to_pandas()
#items['full_text'] = items['title'] + ' ' + items['content'] 
texts = items['title'].to_list()
text_map = dict(zip(items['itemId'].to_list(), texts))

In [4]:
train = pd.read_csv('/kaggle/input/path-to-dzen/train.csv')
train = train.loc[~train.target.isin(['unavailable', 'doubles'])]
train['text_one'] = train['leftItemId'].map(text_map).fillna('Missing') 
train['text_two'] =  train['rightItemId'].map(text_map).fillna('Missing')
train.to_csv('new_train.csv', index=False)
print('Created Train!')

test = pd.read_csv('/kaggle/input/path-to-dzen/test.csv')
test['text_one'] = test['leftItemId'].map(text_map).fillna('Missing') 
test['text_two'] =  test['rightItemId'].map(text_map).fillna('Missing')
test.to_csv('new_test.csv', index=False)
print('Created Test!')

Created Train!
Created Test!


In [26]:
class MyDataset(Dataset):
    def __init__(self, data: str, nrows=40_000):
        self.data = data 

        if self.data == 'train' or self.data == 'valid':
            self.df = pd.read_csv('new_train.csv', nrows=nrows)
            train_df, valid_df = train_test_split(self.df, test_size=0.125, random_state=42)
            
            self.df = train_df if self.data == 'train' else valid_df
        else:
            self.df = pd.read_csv('new_test.csv', nrows=nrows)
         
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text_one = row['text_one']
        text_two = row['text_two']
        
        if self.data == 'train' or self.data == 'valid': 
            label = dict_[row['target']]
            return (text_one, text_two, torch.tensor(label, dtype=torch.long))
        else:
            return (text_one, text_two)

In [38]:
data_train = MyDataset('train')
data_valid = MyDataset('valid')

data_train = DataLoader(data_train, 16, shuffle=True)
data_valid = DataLoader(data_valid, 16, shuffle=False)

In [39]:
data_test = MyDataset('test', nrows=None)
data_test = DataLoader(data_test, 16, shuffle=False)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn 

MODEL_NAME = "DeepPavlov/rubert-base-cased-sentence"

class CustomBert(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.model_bert = AutoModel.from_pretrained(MODEL_NAME, use_safetensors=True)
        self.pooler = nn.Sequential(
            nn.Linear(768, 768),
            nn.Tanh()
        )
        self.fc1 = nn.Linear(768, num_classes) # Финальный слой

    def forward(self, x1):
        outputs = self.model_bert(**x1)
        pooled_output = outputs.last_hidden_state[:, 0]
        return self.fc1(self.pooler(pooled_output))

In [31]:
import torch.nn.functional as F

device = 'cuda:0'

model = CustomBert()
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

EPOCHS = 2

best_score = 1000
early_stopp = 1
now_num = 0 

for i in range(EPOCHS):
    print('='*50)
    print(f"EPOCH: {i+1}")
    print('='*50)
    
    train_sum = 0
    
    model.train()

    for text_one, text_two, label_batch in tqdm(data_train):
        text_new = tokenizer(text_one, text_two, return_tensors='pt', padding=True, truncation=True,max_length=128)
        text_new = text_new.to(device)
        
        label_batch = label_batch.to(device)

        optimizer.zero_grad()

        outputs = model(text_new)

        loss = loss_fn(outputs, label_batch)
        loss.backward()
        optimizer.step()

        train_sum += loss.item()

    print(f'Train Loss: {train_sum / len(data_train)}')

    model.eval()

    val_sum = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for text_one, text_two, label_batch in tqdm(data_valid):
            text_new = tokenizer(text_one, text_two, return_tensors='pt', padding=True, truncation=True,max_length=128)
            text_new = text_new.to(device)
            
            label_batch = label_batch.to(device)
            outputs = model(text_new)
            val_sum += loss_fn(outputs, label_batch).item()
       
            for x in outputs.cpu().numpy(): 
                y_pred.append(np.argmax(x))

            for x in label_batch.cpu().numpy():
                y_true.append(x)

    score = f1_score(y_true, y_pred, average='weighted')
    val_sum = val_sum / len(data_valid)
    print(f'Val Loss: {val_sum}, Metric: {score}')

    if val_sum < best_score:
        best_score = val_sum 
        torch.save(model, 'best_model.pth')
        now_num = 0 
    else:
        now_num += 1 

        if now_num == early_stopp:
            break

EPOCH: 1


100%|██████████| 2188/2188 [04:31<00:00,  8.05it/s]


Train Loss: 1.1000147822297688


100%|██████████| 313/313 [00:10<00:00, 31.16it/s]


Val Loss: 1.0775805667947276, Metric: 0.4829098877302642
EPOCH: 2


100%|██████████| 2188/2188 [04:31<00:00,  8.06it/s]


Train Loss: 0.9090923106321687


100%|██████████| 313/313 [00:10<00:00, 31.21it/s]

Val Loss: 1.1030052460420627, Metric: 0.5037312709698425


In [32]:
dict_reverse = {v: k for k, v in dict_.items()}

In [33]:
dict_reverse

{0: 'no_relevant', 1: 'relevant_minus', 2: 'relevant', 3: 'relevant_plus'}

In [41]:
test = pd.read_csv('new_test.csv')
y_pred = []

with torch.no_grad():
    for text_one, text_two in tqdm(data_test):
        text_batch = tokenizer(text_one, text_two, return_tensors='pt', padding=True, 
                               truncation=True, max_length=128)
        text_batch = text_batch.to(device)
        outputs = model(text_batch)
        
        for x in outputs.cpu().numpy():
            y_pred.append(np.argmax(x))

y_pred = [dict_reverse[xi] for xi in y_pred]
submission = pd.DataFrame({
    'target': y_pred
})

submission.to_csv('submission.csv')
submission

100%|██████████| 3228/3228 [01:42<00:00, 31.63it/s]


target
0            relevant
1            relevant
2       relevant_plus
3         no_relevant
4      relevant_minus
...               ...
51631     no_relevant
51632        relevant
51633        relevant
51634  relevant_minus
51635        relevant

[51636 rows x 1 columns]

In [ ]:
# Unnamed: 0

In [42]:
submission

target
0            relevant
1            relevant
2       relevant_plus
3         no_relevant
4      relevant_minus
...               ...
51631     no_relevant
51632        relevant
51633        relevant
51634  relevant_minus
51635        relevant

[51636 rows x 1 columns]